Principal Component Analysis (PCA) is a linear transformation that transforms data points into feature space representation. It has the following properties:

* Exactly the same dimension as the original data space
* Reconstruction of data points is optimal in the mean square error sense
* Principal components are sorted by "effectiveness" at capturing variance
* Dimensionality reduction can be done simply by truncating principal components

http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html